In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import re
%matplotlib inline

In [2]:
df = pd.read_csv('ps4.csv', header=0, sep=',') 

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2073 entries, 0 to 2072
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            2073 non-null   object
 1   release         2073 non-null   object
 2   genres          2073 non-null   object
 3   metascore       2073 non-null   int64 
 4   users_score     2073 non-null   object
 5   platforms       1829 non-null   object
 6   publisher       2073 non-null   object
 7   num_of_players  1546 non-null   object
 8   rating          1948 non-null   object
 9   developer       2073 non-null   object
dtypes: int64(1), object(9)
memory usage: 162.1+ KB


,name,release,genres,metascore,users_score,platforms,publisher,num_of_players,rating,developer
0,Red Dead Redemption 2,26-Oct-18,"['Action Adventure', 'Open-World']",97,8.6,"['PC', 'Xbox One']",Rockstar Games,Up to 32,M,Rockstar Games
1,Grand Theft Auto V,18-Nov-14,"['Action Adventure', 'Modern', 'Open-World']",97,8.4,"['PC', 'Xbox One']",Rockstar Games,Up to 30,M,Rockstar North
2,Persona 5 Royal,31-Mar-20,"['Role-Playing', 'Japanese-Style']",95,8.4,NaN,Atlus,No Online Multiplayer,M,Atlus
3,The Last of Us Remastered,29-Jul-14,"['Action Adventure', 'General', 'Modern']",95,9.2,NaN,SCEA,Up to 8,M,Naughty Dog
4,God of War,20-Apr-18,"['Action Adventure', 'Linear']",94,9.2,['PC'],Sony Interactive Entertainment,No Online Multiplayer,M,SCE Santa Monica


In [4]:
df.duplicated(keep='first').sum()

0

In [5]:
df['platforms'].fillna('Exclusive',inplace=True)
df['num_of_players'].fillna('Single Player',inplace=True)
df['rating'].fillna('Unknown',inplace=True)

In [6]:
df['users_score']=pd.to_numeric(df['users_score'],errors='coerce')

In [9]:
df['num_of_players']=df['num_of_players'].replace('No Online Multiplayer','Single Player')
df['num_of_players']=df['num_of_players'].replace('1 Player','Single Player')


In [73]:
for i in df['num_of_players']:
    if i != 'Single Player':
        df['num_of_players'] = df['num_of_players'].replace([i],'Multiplayer')

In [74]:
df=pd.get_dummies(df,columns=['num_of_players'],prefix='is')
df['release']=pd.to_datetime(df['release'])
df['month']=df['release'].dt.month
df['year']=df['release'].dt.year

In [75]:
df.drop(['is_Single Player'],axis=1,inplace=True)

In [76]:
df['rating'].unique()
rate_map={"Unknown":0 ,"E":1 ,"E10+":2 ,"T":3 ,"M":4}
df.replace(rate_map,inplace=True)


In [77]:
df['users_score'].fillna(df['users_score'].median(),inplace=True)
df['users_score']=df['users_score']*10


In [78]:
mycols=['metascore','users_score']
outliers=[]
for col in mycols:        
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        IQR = Q3 - Q1
        IQR_range = 1.5 * IQR
        outliers.append(df[col].iloc[:][(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR )])

print(outliers)

[2023    43
2024    43
2025    43
2026    43
2027    43
2028    43
2029    43
2030    43
2031    43
2032    42
2033    42
2034    42
2035    42
2036    42
2037    41
2038    41
2039    41
2040    41
2041    41
2042    40
2043    40
2044    39
2045    39
2046    38
2047    38
2048    38
2049    38
2050    38
2051    38
2052    37
2053    37
2054    37
2055    37
2056    37
2057    36
2058    36
2059    36
2060    36
2061    35
2062    35
2063    33
2064    32
2065    32
2066    31
2067    30
2068    30
2069    28
2070    27
2071    26
2072    21
Name: metascore, dtype: int64, 199     35.0
239     21.0
271     34.0
292     37.0
310     28.0
        ... 
2068    17.0
2069    18.0
2070    28.0
2071    24.0
2072    29.0
Name: users_score, Length: 103, dtype: float64]


In [79]:
df['developer'].replace(["EA Sports","EA DICE", "EA Canada","EA Vancouver","EA Tiburon"], 'EA',inplace=True)
df['developer'].replace('Rockstar North','Rockstar Games',inplace=True)


In [80]:
for i in df['platforms']:
    if i != 'Exclusive':
        df['platforms'] = df['platforms'].replace([i],'Not Exclusive')
df=pd.get_dummies(df,columns=['platforms'],prefix='is')
df.drop(['is_Not Exclusive'],axis=1,inplace=True)

In [85]:
df['is_best']=np.where(df['metascore']>=80,1,0)

In [86]:
df.to_csv('ps4_clean.csv', encoding='utf-8',index=False)